# Evaluation Technique Jimini

## Présentation de Jimini

Pour rappel, nous développons trois outils :
- Un outil d'analyse factuelle de documents;
- Un outil de recherche juridique;
- Un outil de rédaction de documents juridiques, type contrats.

## Information Retrieval

### Objectif

Les deux premiers outils nécessitent d'être capable de récupérer le contexte pertinent dans un ensemble de paragraphes donné - problématique connue sous le nom d'`information retrieval` ou de `document retrieval`. L'objectif de cette partie est donc de construire un moteur de recherche juridique. Nous disposons d'un corpus de documents juridiques, et nous souhaitons que l'utilisateur puisse poser une question en langage naturel, puis que le moteur de recherche lui renvoie les documents les plus pertinents.

### Dataset

Dans le cadre de ce notebook, nous nous intéressons à un jeu de données juridique francophone : [Belgian Statuatory Article Retrieval Dataset (BSARD)](https://huggingface.co/datasets/maastrichtlawtech/bsard), qui contient ~1000 questions posées par les utilisateurs d'une plateforme d'information juridique belge (Droits Quotidiens), ainsi que les articles pertinents pour y répondre. Ce jeu de données est issu de l'article [A Statutory Article Retrieval Dataset in French](https://arxiv.org/pdf/2108.11792.pdf) de l'Université de Maastricht.

Ainsi que détaillé sur la page HuggingFace, le dataset se présente sous la forme de deux fichiers CSV contenant les questions d'entraînement `questions_fr_train.csv` et de test `questions_fr_test.csv`, et structurés comme suit :
 - `id` : un attribut int32 correspondant à un numéro d'identifiant unique pour la question.
 - `question` : un attribut de type chaîne de caractères correspondant à la question.
 - `category` : un attribut de type chaîne de caractères correspondant au sujet général de la question.
 - `subcategory` : un attribut de type chaîne de caractères correspondant au sous-sujet de la question.
 - `extra_description` : un attribut de type chaîne de caractères correspondant aux tags de catégorisation supplémentaires de la question.
 - `article_ids` : un attribut de type chaîne de caractères contenant les ID d'articles pertinents pour la question, séparés par des virgules.

(À noter, ce dataset contient également un échantillon de ~113k questions synthétiques, ainsi que des exemples de *négatifs* durs, c'est-à-dire des questions avec des articles non-pertinents, dont vous pouvez librement vous servir pour entraîner votre modèle.)

Et d'un fichier `articles.csv` :
 - `id` : un attribut int32 correspondant à un numéro d'identifiant unique pour l'article.
 - `article` : un attribut de type chaîne de caractères correspondant à l'intégralité de l'article.
 - `code` : un attribut de type chaîne de caractères correspondant au code de loi auquel appartient l'article.
 - `article_no` : un attribut de type chaîne de caractères correspondant au numéro de l'article dans le code.
 - `description` : un attribut de type chaîne de caractères correspondant aux titres concaténés de l'article.
 - `law_type` : un attribut de type chaîne de caractères dont la valeur est soit "regional" soit "national".

### Évaluation

Ainsi que dans l'[article](https://arxiv.org/pdf/2108.11792.pdf), nous évaluerons la performance du modèle en utilisant les métriques R@100, R@200, R@500, MAP@100 et MRR@100.

Cependant, en conditions réelles, nous souhaitons que le moteur de recherche renvoie les documents les plus pertinents en premier. Nous utiliserons donc la métrique NDCG@100, qui prend en compte l'ordre des documents renvoyés, ainsi que les métriques précédentes, avec un $k$ plus petit (par exemple $k=1$, $k=5$ et $k=10$).

#### Pyterrier

On se sert ici du framework Pyterrier qui permet assez simplement de prototyper et évaluer un moteur de recherche, mais **libre au candidat d'utiliser l'approche qu'il préfère**.
Dans le code suivant, on se sert des notations de `PyTerrier`, qui fonctionne avec les clés suivantes :
- `qid` : `str` l'identifiant de la question
- `docno` : `str` l'identifiant de l'article
- `query` : `str` le texte de la question
- `rel_docnos` : `List[str]` la liste des documents pertinents pour la question
- `score`: `float` le score de pertinence du document pour la question
- `label`: `int` la pertinence du document pour la question (1 si pertinent, 0 sinon)
- `qrels` : `Dict[str, Dict[str, int]]` un dictionnaire contenant les documents pertinents pour chaque question


In [1]:
!pip install datasets==2.15.0 --quiet
!pip install pandas==2.1.3 --quiet
!pip install python-terrier==0.10.0 --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 30.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.1.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py)

In [ ]:
# Retrieve the corpus of articles from the dataset
!wget https://huggingface.co/datasets/maastrichtlawtech/bsard/resolve/main/articles.csv --quiet


In [2]:
import os
import ast
import shutil
import pandas as pd
from datasets import load_dataset
import pyterrier as pt
from pyterrier.measures import *
import torch
from tqdm import tqdm

if not pt.started():
    pt.init()


terrier-assemblies 5.8 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8



In [ ]:
!pip install --upgrade datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 2.15.0
    Uninstalling datasets-2.15.0:
      Successfully uninstalled datasets-2.15.0


In [3]:
from datasets import load_dataset
import pandas as pd

train_file_path = 'questions_train.csv'
test_file_path = 'questions_test.csv'
articles_file_path='articles.csv'


df_train = pd.read_csv(train_file_path)
df_test = pd.read_csv(test_file_path)
df_articles=pd.read_csv(articles_file_path)

# Affichage des premières lignes pour vérifier
print("Train DataFrame:")
print(df_train.head())

print("\nTest DataFrame:")
print(df_test.head())


print("\nArticles DataFrame:")
print(df_articles.head())



Train DataFrame:
     id  category                      subcategory  \
0  1102   Travail           Travail et parentalité   
1    91    Argent                           Dettes   
2   474   Famille             Situation de couples   
3   836  Logement             Location en Wallonie   
4  1079   Travail  Maladie - incapacité de travail   

                                            question  \
0  Je suis travailleur salarié(e). Puis-je refuse...   
1                  Peut-on saisir tous mes revenus ?   
2  Je suis marié(e). Nous sommes mariés. Dois-je ...   
3  Je mets un kot en location (bail de droit comm...   
4  Suis-je payé pendant la procédure du trajet de...   

                                   extra_description  \
0                               Pendant la grossesse   
1  Procédures de récupération des dettes, Récupér...   
2                                            Mariage   
3  Mettre un logement en location (Wallonie), Doi...   
4  Rupture du contrat de travail pour for

In [4]:
df_negative=pd.read_json("negatives_bm25_negatives_train.json")
print(df_negative.shape)

(10, 886)


In [9]:
print(df_articles.shape)

(22633, 12)


In [5]:
# Ensure that the article IDs are strings, and rename the column to 'docno', because of PyTerrier's expectations
df_articles['docno'] = df_articles['id'].astype(str)
df_articles["article"] = df_articles["article"].str.replace("[^a-zA-Z0-9 À-ÿ]", " ", regex=True).str.strip().str.replace(" +", " ", regex=True)

def prepare_queries(df):
    queries = pd.DataFrame({
        'qid': df['id'].astype(str),
        'query': df['question'].str.replace("[^a-zA-Z0-9 À-ÿ]", " ", regex=True).str.strip().str.replace(" +", " ", regex=True),
        'rel_docnos': df['article_ids'].apply(lambda x: list(ast.literal_eval(x)) if isinstance(x, str) and x.startswith('(') else [x])
    })
    return queries


# Prepare the train and test query dataframes
train_queries = prepare_queries(df_train)
test_queries = prepare_queries(df_test)



In [11]:
print(test_queries.head())


    qid                                              query  \
0   775    Quels sont les critères communaux d insalubrité   
1  1053  A t on droit à l allocation de naissance en ca...   
2    72  Quels frais peut on ajouter lors d un recouvre...   
3   861       Comment se déroule une expulsion à Bruxelles   
4   786            Peut on m expulser en hiver en Wallonie   

                                   rel_docnos  
0                                     [12024]  
1                                   [947,948]  
2                            [2041,2119,2138]  
3                  [5562,5563,5564,5565,5566]  
4  [5561,5562,5563,5564,5565,5566,5567,12124]  


In [6]:
qrels = pd.DataFrame([
    {'qid': str(qid), 'docno': str(docno), 'label': 1}
    for qid, docnos in test_queries[['qid', 'rel_docnos']].itertuples(index=False)
    for docno in docnos
])


In [13]:
print(qrels.head)

<bound method NDFrame.head of       qid                                     docno  label
0     775                                     12024      1
1    1053                                   947,948      1
2      72                            2041,2119,2138      1
3     861                  5562,5563,5564,5565,5566      1
4     786  5561,5562,5563,5564,5565,5566,5567,12124      1
..    ...                                       ...    ...
217   524                            2328,2329,2330      1
218    69                                      2127      1
219   463                                      1073      1
220   880                                       857      1
221   512                                 6531,6598      1

[222 rows x 3 columns]>


On indexe les articles dans l'index PyTerrier (`DFIndex` avait un bug, donc on passe par un `IterDictIndex`)

In [7]:
# Indexing the articles using IterDictIndexer
if os.path.exists("./bsard_index"):
    shutil.rmtree("./bsard_index")
indexer = pt.IterDictIndexer("./bsard_index", overwrite=True)
docs = [{"docno": str(row["id"]), "text": row["article"]} for _, row in df_articles.iterrows()]
indexref = indexer.index(docs, fields=["text"])

index = pt.IndexFactory.of(indexref)


18:55:17.623 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (18417) - further warnings are suppressed
18:55:22.937 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 1 empty documents


Le `retriever` doit être capable, étant donné une question, de classer les articles par pertinence. Le format de sortie est un CSV avec trois colonnes :
- `qid` : `str` l'identifiant de la question
- `docno` : `str` l'identifiant de l'article
- `score` : `float` le score de pertinence de l'article pour la question

Libre au candidat de choisir l'approche qu'il préfère pour construire le `retriever`, l'essentiel étant qu'il construise un fichier CSV avec les colonnes `qid`, `docno` et `score`.

**Remarque** : Il peut ainsi y avoir plusieurs lignes pour une même question, si plusieurs articles sont pertinents (ce qui est souvent le cas)

In [ ]:
# Define the retrieval model : for example dummy BM25 and TF-IDF
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF", properties={"c": 1.0})
bm25 = pt.BatchRetrieve(index, wmodel="BM25",  properties={"c": 1.0, "bm25.k_1": 1, "bm25.b": 0.6})

# Run the TF-IDF model and save the results
tfidf_results = tfidf.transform(test_queries)
tfid_results = tfidf_results[["qid", "docno", "score"]]
tfidf_results.to_csv("tfidf_results.csv", index=False)

# Run the BM25 model and save the results
bm25_results = bm25.transform(test_queries)
bm25_results = bm25_results[["qid", "docno", "score"]]
bm25_results.to_csv("bm25_results.csv", index=False)


In [ ]:
# Define the evaluation metrics
eval_metrics = ["recall_1", "recall_5", "recall_10", RR@10, AP@10, "recall_100", "recall_200", "recall_500", "map", "ndcg", RR@100, nDCG@100, AP@100]

# Load the results
tfidf_results = pd.read_csv("tfidf_results.csv")
bm25_results = pd.read_csv("bm25_results.csv")

# Evaluate the models
result = pt.Experiment(
    [tfidf_results, bm25_results],
    test_queries,
    qrels,
    eval_metrics,
    names=['TF-IDF', "BM25"],
)

print(result)


     name  recall_1  recall_5  recall_10     RR@10     AP@10  recall_100  \
0  TF-IDF  0.040541  0.085586   0.112613  0.059032  0.059032    0.216216   
1    BM25  0.027027  0.049550   0.067568  0.037925  0.037925    0.162162   

   recall_200  recall_500       map      ndcg    RR@100  nDCG@100    AP@100  
0    0.243243    0.256757  0.063303  0.100965  0.063036  0.092742  0.063035  
1    0.202703    0.220721  0.041664  0.074317  0.041291  0.063741  0.041291  


## Embeddings des articles et des questions d'entrainements

In [8]:
from transformers import CamembertTokenizer, CamembertModel
import torch

tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
model = CamembertModel.from_pretrained('camembert-base')



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [9]:
def text_to_embedding(description, article, tokenizer, model):
    # on remplace NaN par une chaîne vide
    description = str(description) if not pd.isna(description) else ""
    article = str(article) if not pd.isna(article) else ""
    # Concaténation de la description avec l'article
    combined_text = description + " " + article
    # Tokenize le texte combiné et prépare les tensors
    inputs = tokenizer(combined_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    # Obtention des sorties du modèle
    with torch.no_grad():
        outputs = model(**inputs)
    # dernier état caché
    embeddings = outputs.last_hidden_state.mean(dim=1)

    return embeddings



In [10]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [11]:
def save_embeddings(batch_embeddings, batch_index, save_dir='/content/drive/My Drive/Jimini/article_embeddings'):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    file_path = os.path.join(save_dir, f'embeddings_batch_{batch_index}.pt')
    torch.save(batch_embeddings, file_path)

In [ ]:
df_articles['description'] = df_articles['description'].astype(str)
df_articles['article'] = df_articles['article'].astype(str)


#### Première tentative et echec de l'embedding des articles

In [ ]:
batch_size = 1000

for i in range(0, len(df_articles), batch_size):
    batch = df_articles.iloc[i:i+batch_size]
    batch_embeddings = torch.stack([
        text_to_embedding(row['description'], row['article'], tokenizer, model)
        for _, row in batch.iterrows()
    ])
    save_embeddings(batch_embeddings, batch_index=i//batch_size)
    print(f'Lot {i//batch_size} traité et sauvegardé.')

#8 minutes pour 1000 articles :   176 minutes = 3 heures  pour 22 000 articles

Lot 0 traité et sauvegardé.
Lot 1 traité et sauvegardé.
Lot 2 traité et sauvegardé.
Lot 3 traité et sauvegardé.
Lot 4 traité et sauvegardé.
Lot 5 traité et sauvegardé.
Lot 6 traité et sauvegardé.
Lot 7 traité et sauvegardé.
Lot 8 traité et sauvegardé.
Lot 9 traité et sauvegardé.
Lot 10 traité et sauvegardé.
Lot 11 traité et sauvegardé.
Lot 12 traité et sauvegardé.
Lot 13 traité et sauvegardé.
Lot 14 traité et sauvegardé.
Lot 15 traité et sauvegardé.
Lot 16 traité et sauvegardé.
Lot 17 traité et sauvegardé.
Lot 18 traité et sauvegardé.
Lot 19 traité et sauvegardé.
Lot 20 traité et sauvegardé.
Lot 21 traité et sauvegardé.


In [ ]:
save_dir = '/content/drive/My Drive/Jimini/article_embeddings'  # Le dossier où sont stockés vos embeddings
files = [os.path.join(save_dir, f) for f in os.listdir(save_dir) if f.startswith('embeddings_batch_')]
files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))  # Assurez-vous que les fichiers sont bien triés

for file_path in files:
    # Chargez les embeddings du lot actuel
    batch_embeddings = torch.load(file_path)
    all_article_embeddings.append(batch_embeddings)

# Concaténez tous les embeddings en un seul tensor
all_article_embeddings = torch.cat(all_article_embeddings, dim=0)

final_embeddings_path = os.path.join(save_dir, 'all_article_embeddings.pt')
torch.save(all_article_embeddings, final_embeddings_path)
print(f'Tous les embeddings ont été regroupés et sauvegardés dans {final_embeddings_path}')


#all_embeddings = torch.load(final_embeddings_path)

Tous les embeddings ont été regroupés et sauvegardés dans /content/drive/My Drive/Jimini/article_embeddings/all_article_embeddings.pt


In [ ]:
save_dir = '/content/drive/My Drive/Jimini/article_embeddings'
final_embeddings_path = os.path.join(save_dir, 'all_article_embeddings.pt')
all_embeddings = torch.load(final_embeddings_path)

KeyboardInterrupt: 

In [ ]:
print(all_embeddings.shape)
print(df_articles.shape)

torch.Size([21018, 1, 768])
(22633, 13)


#### Nouvelle tentative de l'embedding des articles

In [12]:
from tqdm import tqdm #pour visualiser l'avancement

save_dir = '/content/drive/My Drive/Jimini/article_embeddings'
num_articles = len(df_articles)
embedding_dim = 768  # dimension de sortie d'un Bert

def text_to_embedding_with_fallback(description, article, tokenizer, model):
    try:
        return text_to_embedding(description, article, tokenizer, model)
    except Exception as e:
        print(f"Error generating embedding: {e}")
        return torch.zeros((1, embedding_dim))  # j'ai recontré des erreurs pendant l'embedding donc je retourne des 0 dans ce cas



for i in range(0, num_articles, 10000):
    final_embeddings_path = os.path.join(save_dir, f'all_article_embeddings_{i}.pt')
    # je regarde si l'embedding existe deja
    if i==10000: # j'avais deja enregistré les autres, je voulais refaire que celui la
      if not os.path.exists(final_embeddings_path):
          batch_size = min(10000, num_articles - i)  # ajustement de batch size pour le dernier batch
          print(f"Generating embeddings for articles {i} to {i + batch_size}")
          batch_embeddings = torch.zeros((batch_size, embedding_dim))
          for j, row in tqdm(enumerate(df_articles[i:i+batch_size].iterrows()), total=batch_size):
              _, row = row  #(index, Series)
              embedding = text_to_embedding_with_fallback(row['description'], row['article'], tokenizer, model).squeeze()
              batch_embeddings[j] = embedding

          # Sauvegarder les  embeddings apres chaque batch
          torch.save(batch_embeddings, final_embeddings_path)
          print(f'Embeddings saved in {final_embeddings_path}')
      else:
          print(f'Embeddings file {final_embeddings_path} already exists. Skipping generation.')



Embeddings file /content/drive/My Drive/Jimini/article_embeddings/all_article_embeddings_10000.pt already exists. Skipping generation.


In [ ]:
e1=torch.load("/content/drive/My Drive/Jimini/article_embeddings/all_article_embeddings_0.pt")
e2=torch.load("/content/drive/My Drive/Jimini/article_embeddings/all_article_embeddings_10000.pt")
e3=torch.load("/content/drive/My Drive/Jimini/article_embeddings/all_article_embeddings_20000.pt")
e1=e1[:10000] # e1 de la mauvaise taille

#print(e1[9999])
print(e2.shape)
print(e3.shape)
print(len(df_articles))

# Concaténation des tensors pour former un seul tensor d'embeddings
final_article = torch.cat([e1, e2, e3], dim=0)
#print(final_article[9999])
print(final_article.shape)
torch.save(final_article, "/content/drive/My Drive/Jimini/article_embeddings/final_article.pt")

torch.Size([10000, 768])
torch.Size([2633, 768])
22633
torch.Size([22633, 768])


Approche naive avec la similarité cosinus

In [97]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Générer des embeddings pour quelques questions et articles
question_embeddings = torch.cat([text_to_embedding("",df_test['question'].iloc[3], tokenizer, model)])
articles_embeddings = torch.cat([
    text_to_embedding(row['description'], row['article'], tokenizer, model)
    for _, row in df_articles.iloc[5400:5600].iterrows()
])
# 20 sec pour 50 embeddings :
# donc 9 000 secs pour 22 000 embeddings :


# Supprimez la dimension inutile de taille 1 pour les deux ensembles d'embeddings
question_embedding_np = question_embeddings.squeeze().numpy()
article_embeddings_np = articles_embeddings.squeeze().numpy()

# Maintenant, calculez la similarité cosinus comme prévu
similarities = cosine_similarity(question_embedding_np.reshape(1, -1), article_embeddings_np)

# Le reste de votre code pour récupérer les top 5 articles peut rester le même
top5_idx = np.argsort(similarities[0])[-5:]
top5_articles_ids = df_articles.iloc[top5_idx]['id'].values

print("Top 5 des articles les plus similaires :", [top5_articles_ids[i]+5400 for i in range(len(top5_articles_ids))])
# devrait etre 5562

Top 5 des articles les plus similaires : [875, 1033, 874, 1024, 1029]


In [ ]:
print(all_train_embeddings.shape)
print(df_train.shape)


final_embeddings_path = os.path.join(save_dir, 'all_train_embeddings.pt')
torch.save(all_train_embeddings, final_embeddings_path)
print(f'Tous les embeddings ont été regroupés et sauvegardés dans {final_embeddings_path}')



torch.Size([886, 768])
(886, 6)
Tous les embeddings ont été regroupés et sauvegardés dans /content/drive/My Drive/Jimini/article_embeddings/all_train_embeddings.pt


## Chargement des embeddings et données

In [13]:
all_train_embeddings=torch.load('/content/drive/My Drive/Jimini/article_embeddings/all_train_embeddings.pt' )
all_article_embeddings=torch.load('/content/drive/My Drive/Jimini/article_embeddings/final_article.pt' )

In [14]:
import json

# Remplacez 'path_to_your_file.json' par le chemin réel de votre fichier JSON
with open('negatives_bm25_negatives_train.json', 'r') as f:
    negative_map = json.load(f)

negative_map = {int(k): v for k, v in negative_map.items()}

positive_map = {int(row['id']): [int(article_id.strip()) for article_id in str(row['article_ids']).split(',')] for _, row in df_train.iterrows()}



## Optimisation et entrainement

In [15]:
import random
from torch.utils.data import Dataset, DataLoader

class QuestionArticleDataset(Dataset):
    def __init__(self, questions, articles, positive_map, negative_map):
        self.questions = questions
        self.articles = articles
        # S'assurer que seules les clés présentes à la fois dans positive_map et negative_map sont utilisées
        self.keys = [key for key in positive_map.keys() if key in negative_map]
        self.positive_map = positive_map
        self.negative_map = negative_map

    def __len__(self):
        # Utiliser la longueur de self.keys pour déterminer le nombre d'éléments dans le Dataset
        return len(self.keys)

    def __getitem__(self, idx):
        # Utiliser self.keys[idx] pour obtenir l'ID de la question actuelle
        question_id = self.keys[idx]

        # Utiliser question_id pour accéder à positive_map et negative_map
        q_embedding = self.questions[idx]
        a_pos_index = random.choice(self.positive_map[question_id])  # Choix aléatoire d'un article positif
        a_neg_index = random.choice(self.negative_map[question_id])  # Choix aléatoire d'un article négatif

        a_pos_embedding = self.articles[a_pos_index]
        a_neg_embedding = self.articles[a_neg_index]

        return q_embedding, a_pos_embedding, a_neg_embedding

questions_embeddings = all_train_embeddings
articles_embeddings = all_article_embeddings
positive_map = positive_map
negative_map = negative_map

# Initialiser le Dataset
dataset = QuestionArticleDataset(questions_embeddings, articles_embeddings, positive_map, negative_map)

# Créer un DataLoader
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [123]:

import torch.nn as nn
import torch.optim as optim

class Projector(nn.Module):
    def __init__(self, input_dim, proj_dim):
        super(Projector, self).__init__()
        self.fc = nn.Linear(input_dim, proj_dim)

    def forward(self, x):
        return self.fc(x)

# Initialisation des projecteurs
input_dim = 768  # taille de sortie d'un BERT
proj_dim = 128 # Dimension souhaitée des embeddings projetés
P_q = Projector(input_dim, proj_dim)
P_a = Projector(input_dim, proj_dim)
num_epochs=10
# Fonction de perte et optimiseur
margin = 1
loss_fn = nn.TripletMarginLoss(margin=margin, p=2) # margin garantit que les exemples similaires sont dans une marge de sécurité
optimizer = optim.Adam(list(P_q.parameters()) + list(P_a.parameters()), lr=0.001)

# entrainement
for epoch in range(num_epochs):
    epoch_loss = 0
    for q, a_pos, a_neg in data_loader:
        optimizer.zero_grad()
        q_proj = P_q(q)
        a_pos_proj = P_a(a_pos)
        a_neg_proj = P_a(a_neg)
        loss = loss_fn(q_proj, a_pos_proj, a_neg_proj)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {epoch_loss/len(data_loader)}')


Epoch 1, Loss: 0.8497014088290078
Epoch 2, Loss: 0.6886797706995692
Epoch 3, Loss: 0.6008915188057082
Epoch 4, Loss: 0.5715664880616325
Epoch 5, Loss: 0.5482889626707349
Epoch 6, Loss: 0.513806477189064
Epoch 7, Loss: 0.4958223540868078
Epoch 8, Loss: 0.457973803792681
Epoch 9, Loss: 0.4020880439451763
Epoch 10, Loss: 0.3865130660789354


In [127]:
save_path = '/content/drive/My Drive/Jimini/article_embeddings/P_q_projector_ex.pth'

# Enregistrement du modèle
torch.save(P_q.state_dict(), save_path)

In [124]:
P_a.eval()

# Appliquez P_a à tous les embeddings
with torch.no_grad():
    projected_articles = P_a(all_article_embeddings)

# Chemin pour sauvegarder les embeddings projetés sur Google Drive
projected_embeddings_path = os.path.join('/content/drive/My Drive/Jimini/article_embeddings', 'all_projected_article_embeddings_ex.pt')

# Sauvegarder sur Google Drive
torch.save(projected_articles, projected_embeddings_path)
print(f'Les embeddings projetés ont été sauvegardés dans {projected_embeddings_path}')


Les embeddings projetés ont été sauvegardés dans /content/drive/My Drive/Jimini/article_embeddings/all_projected_article_embeddings_ex.pt


## Trouver l'article correpondant

charger les articles projettés et le projecteur de question

In [22]:
# Création d'une nouvelle instance de projecteur
P_q_loaded = Projector(input_dim, proj_dim)

# Chargement des paramètres enregistrés
P_q_loaded.load_state_dict(torch.load('/content/drive/My Drive/Jimini/article_embeddings/P_q_projector_ex.pth'))

# Assurez-vous de passer le modèle en mode évaluation si vous l'utilisez pour des inférences
P_q_loaded.eval()

projected_articles=torch.load('/content/drive/My Drive/Jimini/article_embeddings/all_projected_article_embeddings_ex.pt')
print(projected_articles.shape)

Projector(
  (fc): Linear(in_features=768, out_features=128, bias=True)
)

In [126]:
# Pour une question, trouver le plus proche voisin
from scipy.spatial import distance
import numpy as np



def find_closest_articles(question_text, description, projected_articles, tokenizer, model, projector, n=10):
    # Concaténation de la description avec la question
    combined_text = description + " " + question_text

    # Tokenizer le texte combiné
    inputs = tokenizer(combined_text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Obtention des sorties du modèle
    with torch.no_grad():
        outputs = model(**inputs)

    # Utilisation des embeddings du dernier état caché et projeter
    embeddings = outputs.last_hidden_state.mean(dim=1)
    projected_q = projector(embeddings)

    # réduction des dimensions
    projected_q = projected_q.detach().squeeze()  # Réduire à un vecteur 1-D

    # Calculer les distances
    distances = []
    for i, projected_a in enumerate(projected_articles):
        projected_a = projected_a.squeeze()  # vecteur 1-D
        dist = distance.euclidean(projected_q.numpy(), projected_a.numpy())
        distances.append((dist, i))

    # Trouver les 10 plus petites distances
    distances.sort(key=lambda x: x[0])
    closest_ids = [distances[i][1] for i in range(n+5)]
    closest_dist=[[distances[i][0] for i in range(n+5)]]
    return closest_ids, closest_dist




# Trouver les IDs des articles les plus proches

print ("TRAIN")
print("\n")
closest_article_ids = find_closest_articles("Je suis travailleur salarié(e). Puis-je refuser de faire des heures supplémentaires ou de travailler de nuit ?", "Travail et parentalité, pendant la grossese", projected_articles, tokenizer, model, P_q)
print("Les IDs des 5 articles les plus proches sont :", closest_article_ids[0])
print("devrait être : [22225,22226,22227,22228,22229,22230,22231,22232,22233,22234]")
print("\n")
closest_article_ids = find_closest_articles("Je suis marié(e). Nous sommes mariés. Dois-je reconnaître mon enfant ?", "Situation de couples; Mariage", projected_articles, tokenizer, model, P_q)
print("Les IDs des 5 articles les plus proches sont :", closest_article_ids[0])
print("devrait être : [1096,1097,1098,1108,1109,1110]")
print("\n")
closest_article_ids = find_closest_articles("Qu'est-ce que le trajet de réintégration ?", "Santé et maladie, protection sociale, Reprise d'un travail adapté pendant l'incapacité, Trajet de réintégration", projected_articles, tokenizer, model, P_q)
print("Les IDs des 5 articles les plus proches sont :", closest_article_ids[0])
print("devrait être : [21114,21115,21116,21117,21118,21119,21120,21121,21122,21123,21124]")
print("\n")
print("\n")


print("TEST")
print("\n")
closest_article_ids = find_closest_articles("Peut-on me confisquer ou saisir ma voiture si on y trouve du cannabis ?", "Argent, dette, Procédures de récupération des dettes, Récupération amiable des dettes", projected_articles, tokenizer, model, P_q)
print("Les IDs des 5 articles les plus proches sont :", closest_article_ids[0])
print("devrait être : [2041,2119,2138]")

print("\n")
closest_article_ids = find_closest_articles("Quels frais peut-on ajouter lors d'un recouvrement amiable ?", "justice, Détenir, acheter et vendre du cannabis", projected_articles, tokenizer, model, P_q)
print("Les IDs des 5 articles les plus proches sont :", closest_article_ids[0])
print("devrait être : [6080,6120,6121,6122,6123,6124,6835,6769]")

for i in range(10):
  print("\n")
  closest_article_ids = find_closest_articles(df_train.iloc[i]['question'], df_train.iloc[i]['category']+df_train.iloc[i]['subcategory'], projected_articles, tokenizer, model, P_q)
  print("Les IDs des 5 articles les plus proches sont :", closest_article_ids[0])
  print("devrait être : ", df_test.iloc[i]["article_ids"])



TRAIN


Les IDs des 5 articles les plus proches sont : [2821, 907, 949, 6721, 22231, 6646, 2826, 1659, 952, 6734, 1153, 6383, 2822, 5468, 934]
devrait être : [22225,22226,22227,22228,22229,22230,22231,22232,22233,22234]


Les IDs des 5 articles les plus proches sont : [1017, 1045, 1084, 1091, 1089, 1080, 1061, 1053, 1018, 1030, 1090, 2447, 2360, 1068, 1019]
devrait être : [1096,1097,1098,1108,1109,1110]


Les IDs des 5 articles les plus proches sont : [823, 1150, 1306, 13912, 13404, 1113, 1305, 6507, 5572, 914, 5423, 1115, 1116, 1502, 1310]
devrait être : [21114,21115,21116,21117,21118,21119,21120,21121,21122,21123,21124]




TEST


Les IDs des 5 articles les plus proches sont : [5308, 2421, 2203, 5491, 2005, 981, 2205, 2804, 2805, 2157, 2787, 1065, 2002, 2162, 2029]
devrait être : [2041,2119,2138]


Les IDs des 5 articles les plus proches sont : [4825, 5162, 5557, 13658, 4821, 5947, 5181, 13914, 4961, 5216, 4822, 4843, 2976, 4766, 6502]
devrait être : [6080,6120,6121,6122,6123,6124,68

## Evaluation du retriever

In [34]:
# inutile donne 0 ici
def give_score(articles1,articles2):
    score=0
    for article in articles1:
      if article in articles2:
        score+=1
    return score/len(articles2)


In [35]:
print(test_queries.head)

<bound method NDFrame.head of       qid                                              query  \
0     775    Quels sont les critères communaux d insalubrité   
1    1053  A t on droit à l allocation de naissance en ca...   
2      72  Quels frais peut on ajouter lors d un recouvre...   
3     861       Comment se déroule une expulsion à Bruxelles   
4     786            Peut on m expulser en hiver en Wallonie   
..    ...                                                ...   
217   524  Je deviens cohabitant légal Dois je payer les ...   
218    69                       A qui dois je payer ma dette   
219   463  Je suis marié e On prend un logement en locati...   
220   880  Est ce que je peux signer plusieurs baux de co...   
221   512  Je suis victime de violences conjugales En tan...   

                                     rel_docnos  
0                                       [12024]  
1                                     [947,948]  
2                              [2041,2119,2138]  
3

In [111]:
results = []

for idx, row in test_queries.iterrows():
    qid = row['qid']
    query_text = row['query']
    n=len(row['rel_docnos'])
    #description = df_test.iloc[idx]["category"]+df_test.iloc[idx]["subcategory"]+ df_test.iloc[idx]["extra_description"] # Ou utilisez une colonne de 'test_queries' si applicable
    description = df_test.iloc[idx]["category"]+df_test.iloc[idx]["subcategory"]
    closest_article_ids = find_closest_articles(query_text, description, projected_articles, tokenizer, model, P_q, n)[0]
    score=give_score(list(closest_article_ids),row['rel_docnos'])
    # Pour cet exemple, les scores sont définis comme inverses des rangs (simple exemple)
    for rank, article_id in enumerate(closest_article_ids):
        results.append({'qid': qid, 'docno': article_id, 'score': score})



In [113]:
print(results_df)
file_path = '/content/drive/My Drive/Jimini/article_embeddings/custom_retrieval_results.csv'
# Enregistrer le fichier CSV
results_df.to_csv(file_path, index=False)


      qid  docno  score
0     775    855    0.0
1     775    856    0.0
2     775   1338    0.0
3     775   6591    0.0
4     775   1502    0.0
...   ...    ...    ...
1327  512  22445    0.0
1328  512  22449    0.0
1329  512   1087    0.0
1330  512   2830    0.0
1331  512   1078    0.0

[1332 rows x 3 columns]


## Quelques questions ouvertes

### Information Retrieval

Dans le cas de *Jimini Analyzer*, on est dans un contexte un peu particulier, où l'information provient de documents structurés.
- L'approche précédente ne tient compte que du contenu des paragraphes, pas de leur place dans la structure du document. Comment pourrait-on améliorer cela ?

- Que pensez-vous du dataset utilisé ? Comment pourrait-on l'améliorer ?
plus de données


- Quelle stratégie adopter pour faire un résumé d'un long document ?


### Raisonnement juridique


Récupérer le contexte pertinent est une étape *sine qua none*, mais il faut ensuite fournir le contexte au LLM, et qu'il réponde à la question posée.
- Comment évaluer le LLM sur le contenu de ses réponses ?
- Quelles métriques utiliser ?
- Quel dataset utiliser / construire pour faire acquérir un raisonnement juridique au LLM ?
- Comment s'assurer que le dataset est de qualité / représentatif ?  
- De quel modèle partir ?
- Quelle technique d'entraînement utiliser ? (full-parameter fine-tuning, PEFT - LoRA - QLoRA , prompt-tuning,...)
- Comment contrôler l'hallucination ?
- Comment faire de l'amélioration continue, en s'assurant que le modèle ne se dégrade pas lorsqu'on lui enseigne une nouvelle compétence ?
- Comment limiter les frais de calculs ?


### Rédaction de documents

- Comment rédiger un contrat de plusieurs dizaines de pages ?

### Bonus

Sentez-vous libre de proposer des améliorations, des idées, des pistes, etc. !


